**About** : This notebook is used to train models.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.cuda.get_device_name(0)
print(device)

In [ ]:
import os
import sys
import glob
import json
import torch
import operator
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
from util.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
#     init_neptune,
)

from params import *
from data.dataset import *
from data.preparation import *
from util.metrics import rsna_loss
from util.plots import plot_confusion_matrix
from model_zoo.models_lvl2 import define_model
from inference.extract_features import Config as ConfigInf
from training.main_lvl2 import k_fold, retrieve_preds

## Data

In [ ]:
df_patient, df_img = prepare_data(DATA_PATH)

In [ ]:
EXP_FOLDERS = [
    ("../logs/2023-09-20/14/", "seg"),
    ("../logs/2023-09-20/36_r/", "probas_2d"),  # 0.358 - convnext-tiny best  <-
    ("../logs/2023-10-05/13/", "probas_2d"),  # 0.353 - maxvit_tiny_tf_384   (+36_r - 0.347)       <- 
    ("../output/confs_rsna_v2_fold*", "yolox"),
#     ("../output/confs_rsna_v1_fold*", "yolox"),
]
EXP_FOLDER = EXP_FOLDERS[0][0]

In [ ]:
config = ConfigInf(json.load(open(EXP_FOLDER + "config.json", "r")))

In [ ]:
if "fold" not in df_patient.columns:
    folds = pd.read_csv(config.folds_file)
    df_img = df_img.merge(folds)
    df_patient = df_patient.merge(folds)

In [ ]:
# for fold in range(4):
#     print('\n Fold', fold)
#     df_imgv = df_img[df_img['fold'] == fold].reset_index()
#     for x in EXP_FOLDERS:
#         if "probas" in x[1]:
#             p = np.load(x[0] + f'pred_val_{fold}.npy')
#             df_imgv['pred'] = p[:, 1]
#         elif "yolox" in x[1]:
#             p = np.load(sorted(glob.glob(x[0]))[fold])
#             df_imgv['pred'] = p
#         else:
#             continue

#         print(x[0][:23], f"  \t {roc_auc_score(df_imgv['extravasation_injury'], df_imgv['pred']):.3f}")

In [ ]:
dataset = PatientFeatureDataset(
    df_patient[df_patient['fold'] == 3],
    df_img[df_img['fold'] == 3],
    EXP_FOLDERS,
    use_other_series=True,
    max_len=600,
    resize=200,
    restrict=True,
)

In [ ]:
idx = np.random.choice(len(dataset))
d = dataset[idx]

In [ ]:
# np.array([
#     dataset.crop_fts[(10004, 21057)],
#     dataset.crop_fts[(10004, 51033)],
# ])

In [ ]:
# lens = []
# for i in tqdm(range(len(dataset))):
#     x = dataset[i][0]
#     lens.append(len(x))

#     if i > 3:
#         break

# sns.histplot(lens)
# plt.show()

In [ ]:
# lens = []
# for i in tqdm(range(len(dataset))):
#     fts = dataset[i][0]
#     x = fts.numpy()
#     lens.append(len(x))
    
# #     start, end = detect_start_end(x)
    
#     if len(x) > 1000:
# #         plt.subplot(1, 2, 1)
#         plt.plot(x[:, :5])
# #         plt.axvline(start, c="salmon")
# #         plt.axvline(end, c="salmon")
        
# #         plt.subplot(1, 2, 2)
# #         plt.plot(kept)
#         plt.show()
    
#     break

In [ ]:
# sns.histplot(lens)

## Model

In [ ]:
x = dataset[0][0]

In [ ]:
# model = define_model(
#     "rnn_att", 
#     ft_dim=x['x'].shape[-1],
#     layer_dim=512,
#     n_layers=1,
#     dense_dim=256,
#     num_classes=11,
#     num_classes_aux=0,
#     n_fts=x['ft'].shape[-1] * x['ft'].shape[-2],
#     use_other_series=True,
# )

In [ ]:
# model(
#     x['x'].unsqueeze(0),
#     x['ft'].unsqueeze(0),
# #     x['other_x'].unsqueeze(0)
# )

## Training
- Handle variable sequence length more cleverly
- Tweak CNN
- Tweak fancier archs

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True

    # Data
    exp_folders = [
        ("../logs/2023-09-20/14/", "seg"),  # v2-rw-t stride+ 384
#         ("../output/confs_rsna_v2_fold*", "yolox"),

#         ("../logs/2023-09-24/20/", "seg3d"),  # resnet18d 3D

        ("../logs/2023-09-20/36_r/", "probas_2d"),  # 0.358 - convnext-tiny best  <-
#         ("../logs/2023-09-27/20_r/", "probas_3d"),  # 0.349 with 36_r - convnext-tiny rnn 3fs3
#         ("../logs/2023-09-26/39_r/", "probas_3d"),  # convnextv2_tiny rnn 2.5D 5fs5
        
#         ("../logs/2023-09-25/39/", "probas_2d"),  # 0.362 convnextv2_tiny bs8
#         ("../logs/2023-10-04/19/", "probas_2d"),  # 0.359 convnextv2_tiny repro new - -0.001 resize longest / +0.002 crop 
        
#         ("../logs/2023-10-05/13/", "probas_2d"),  # 0.353 - maxvit_tiny_tf_384   (+36_r - 0.347)       <- 
        ("../logs/2023-10-10/25/", "probas_2d"),  # 0.352 - maxvit_small_tf_384   <-
#         ("../logs/2023-10-10/27/", "probas_2d"),  # 0.346  - maxvit_tiny_tf_512   <-
        
#         ("../logs/2023-10-06/38/", "probas_2d"),  # 0.361 convnextv2_tiny
#         ("../logs/2023-10-10/31/", "probas_2d"),  # 0.359  - maxvit_small_tf_384
#          ("../logs/2023-10-10/33/", "probas_2d"),  # 0.382 
#         ("../logs/2023-10-06/3/", "probas_3d"),  # 0.357 maxvit_tiny_tf_384 rnn 3fs3
#         ("../logs/2023-10-06/49/", "probas_3d"),  # 0.360 maxvit_tiny_tf_384 rnn pre 3fs3
#         ("../logs/2023-10-06/50/", "probas_3d"),  # 0.367 convnextv2_tiny rnn pre 3fs3

        
        ### CROP MODELS with 3x convnext
        # Convnext
#         ("../logs/2023-09-30/46/", "crop"),  # -> 0.332 +
#         ("../logs/2023-10-03/1/", "crop"),   # -> 0.332 +
        
        # CoAt
#         ("../logs/2023-10-02/11/", "crop"),  # -> 0.328  <- 
#         ("../logs/2023-10-02/36/", "crop"),  # -> 0.329  <- 
#         ("../logs/2023-10-02/41/", "crop"),  # -> 0.327  <- 
#         ("../logs/2023-10-02/51/", "crop"),  # -> 0.329  <- 
#         ("../logs/2023-10-02/70/", "crop"),  # -> 0.328 5x5 (<-)
        
#         ("../logs/2023-10-05/6/", "crop"),  # -> 0.325
#         ("../logs/2023-10-05/8/", "crop"),  # -> 0.328
#         ("../logs/2023-10-05/8/", "crop"),  # coatnet_1_rw_224 -> 0.326  +0.0001
#         ("../logs/2023-10-06/30/", "crop"),  # coat_lite_medium  -> 0.327
        
#         # Bowel / Extrav
#         ("../logs/2023-10-07/4/", "bowel_extrav"),  # convnextv2_tiny
#         ("../logs/2023-10-07/7/", "bowel_extrav"),  # maxvit_tiny_tf_384

#         # CROP MODELS with 2x Convnext & maxvit
        ("../logs/2023-10-02/11/", "crop"),  # coat_lite_medium 3 7         -> 0.326   +0.0015
        ("../logs/2023-10-02/36/", "crop"),  # coat_lite_medium_384 3 7     -> 0.326  +0.0013
        ("../logs/2023-10-02/41/", "crop"),  # coatnet_1_rw_224 3 7         -> 0.325  +0.0008
        ("../logs/2023-10-05/6/", "crop"),   # coatnet_1_rw_224 -1 7        -> 0.322 +0.0008
        ("../logs/2023-10-05/20/", "crop"),  # coatnet_rmlp_1_rw2_224 -1 7  -> 0.325  +0.0010
        ("../logs/2023-10-05/21/", "crop"),  # coat_lite_medium_384 -1 7    -> 0.325  +0.0001
        ("../logs/2023-10-05/31/", "crop"),  # coatnet_1_rw_224 -1 11       -> 0.320  +0.0016
        ("../logs/2023-10-06/31/", "crop"),  # coatnet_rmlp_1_rw2_224 -1 11 -> 0.325  +0.0021

#         # New batch with transfo / lstm_att
# #         ("../logs/2023-10-12/0/", "crop"),  # coatnet_1_rw_224 -1 9 lstm att -> -0.0003
#         ("../logs/2023-10-12/1/", "crop"),  # coatnet_1_rw_224 -1 11 transfo -> -0.0001  - SWAP FOR  05/21 - 0.3109
# #         ("../logs/2023-10-12/4/", "crop"),  # coat_lite_medium transfo -1 11 lstm att -> -0.0001    - SWAP FOR  05/21 - 0.3113
# #         ("../logs/2023-10-12/5/", "crop"),  # coat_lite_medium_384 transfo -1 7 lstm att -> -0.0001    - SWAP FOR  05/21
# #         ("../logs/2023-10-12/7/", "crop"),  # coatnet_rmlp_1_rw2_224 transfo -1 7 lstm att -> -0.0007
#         ("../logs/2023-10-12/8/", "crop"),  # coat_lite_medium transfo -1 11 lstm att -> -0.0001    - SWAP FOR  05/21 0.3110
#         ("../logs/2023-10-12/5/", "crop"),  # coat_lite_medium_384
    ]

    restrict = True
    max_len = 600
    resize = 200
    n_fts = 9 * len([p for p in exp_folders if "crop" in p[1]])
    
    use_other_series = False
    refine_target = False

    # k-fold
    k = 4
    folds_file = f"../input/folds_{k}.csv"
    selected_folds = [0, 1, 2, 3]

    # Model
#     name = "rnn"
    name = "rnn_att"
    ft_dim = (
        22 * len([p for p in exp_folders if "probas" in p[1]]) + 
        4 * len([p for p in exp_folders if "bowel_extrav" in p[1]]) + 
        5
    )
#     if any(["yolox" in mode for _, mode in exp_folders]):
#         ft_dim += 1

    dense_dim = 384  # 384
    layer_dim = 256
    n_layers = 1

    p = 0.
    use_msd = False
    num_classes = 11
    num_classes_aux = 0

    # Training    
    loss_config = {
        "name": "patient",
        "weighted": True,
        "use_any": True,
        "accentuate": False,
        "smoothing": 0,
        "activation": "patient",
        "aux_loss_weight": 0,
        "name_aux": "patient",
        "smoothing_aux": 0,
        "activation_aux": "",
    }

    data_config = {
        "batch_size": 64,
        "val_bs": 256,
        "mix": "mixup",
        "mix_proba": 0.,
        "sched": False,
        "mix_alpha": 4.,
        "additive_mix": False,
        "num_classes": num_classes,
        "num_workers": 8,
    }

    optimizer_config = {
        "name": "AdamW",
        "lr": 4e-4,  # 4e-4, 5e-4
        "warmup_prop": 0.,
        "betas": (0.9, 0.999),
        "max_grad_norm": 10.,
        "weight_decay": 0.,
    }

    epochs = 10

    use_fp16 = True
    verbose = 1
    verbose_eval = 50

    fullfit = False
    n_fullfit = 1

    local_rank = 0
    distributed = False
    world_size = 1

In [ ]:
DEBUG = False
log_folder = None

In [ ]:
# if not DEBUG:
#     log_folder = prepare_log_folder(LOG_PATH)
#     print(f"Logging results to {log_folder}")
#     config_df = save_config(Config, log_folder + "config.json")
#     create_logger(directory=log_folder, name="logs.txt")

# preds, preds_aux = k_fold(Config, df_patient, df_img, log_folder=log_folder, run=None)

### Eval

In [ ]:
# !cat ../logs/2023-10-07/25//config.json
# !cat ../logs/2023-10-07/25//logs.txt

In [ ]:
# !cat ../logs/2023-10-06/41/config.json
# !cat ../logs/2023-10-07/11/logs.txt

In [ ]:
EXP_FOLDERS = [
#     "../logs/2023-10-02/27/",     # 0.344 - rnn_att p - convnext-tiny best 2D & RNN 3 & 5 frames  ---     20 39     <---   LB 0.41
#     "../logs/2023-10-03/22/",   # 0.331  - 20 36 - 41                              <---  LB 0.38
#     "../logs/2023-10-07/26/",   # 0.327   - 13 36 - 41                              <---  LB 0.36
#     "../logs/2023-10-11/3/",   # 0.323   - 13 36 - 41                              <---  LB 0.3??


#     "../logs/2023-09-30/31/",   # 0.334 - rnn_att - convnext-tiny best 2D & RNN 3 & 5 frames & crop   20 39
    
#     "../logs/2023-10-01/28/",   # 0.329 - rnn_att - convnext-tiny best 2D & RNN 3 & 5 frames & crop   20 39 - 20 46 0
#     "../logs/2023-10-01/31/",   # 0.329 - rnn_att - convnext-tiny best 2D & RNN 3 & 5 frames & crop   20 39 - 20 46 0
#     "../logs/2023-10-01/39/",   # 0.329 - rnn_att - convnext-tiny best 2D & RNN 3 & 5 frames & crop   20 39 - 20 46 50 0
#     "../logs/2023-10-01/41/"   # 0.329 - rnn_att - convnext-tiny best 2D & RNN 3 & 5 frames & crop   20 39 - 20 29 46 50 0
    
#     "../logs/2023-10-02/26/",   # 0.326 - rnn_att - convnext-tiny best 2D & RNN 3 & 5 frames & crop   20 39 - 46 11
#     "../logs/2023-10-02/25/",   # 0.325 - rnn_att - convnext-tiny best 2D & RNN 3 & 5 frames & crop   20 39 - 46 11 41
#     "../logs/2023-10-02/25/",   # 0.324 - rnn_att - convnext-tiny best 2D & RNN 3 & 5 frames & crop   20 39 - 46 11 41 51 
    
#     "../logs/2023-10-02/63/",   # 0.326 - rnn_att - convnext-tiny best 2D & RNN 3 & crop   20 - 46 11 36 41 51
#     "../logs/2023-10-02/64/",   # 0.325 - rnn_att - convnext-tiny best 2D & RNN 3 & crop   20 - 11 36 41 51
#     "../logs/2023-10-02/68/",   # 0.327 - rnn_att - convnext-tiny best 2D & RNN 3 & crop   20 - 11 41
#     "../logs/2023-10-03/13/",   # 0.325 - rnn_att - convnext-tiny best 2D & RNN 3 & crop   20 - 11 36 41 51 70

#     "../logs/2023-10-04/1/"

#     "../logs/2023-10-05/11/",  # 0.324 - rnn_att - 36_r 20_r - 11 36 41 51      
#     "../logs/2023-10-05/19/",  # 0.322  -  rnn_att - 36_r 20_r - 11 36 41 51 6
#     "../logs/2023-10-05/19/",  # 0.321  -  rnn_att - 36_r 20_r - 11 36 41 51 6 8
#     "../logs/2023-10-05/49/",  # 0.319 - rnn_att - 36_r 20_r 13 - 41 6 20 21
#     "../logs/2023-10-05/53/",  # 0.321 - rnn_att - 36_r 20_r  - 41 6 20 21
#     "../logs/2023-10-05/54/",  # 0.316 - rnn_att - 36_r 20_r 13 - 11 36 41 6 8 20 21
#     "../logs/2023-10-05/55/",  # 0.318 - rnn_att - 36_r 13 - 11 36 41 6 8 20 21
#     "../logs/2023-10-05/56/",  # 0.316 - rnn_att - 36_r 20_r 13 - 11 36 41 51 70 6 8 20 21

#     "../logs/2023-10-06/13/",  # 0.3144 - rnn_att - 36_r 20_r 13 - 11 36 41 6 8 20 21 31
#     "../logs/2023-10-06/18/",  # 0.3144 - rnn_att - 36_r 20_r 13 - 11 36 41 6 8 20 21 31
#     "../logs/2023-10-06/22/",  # 0.3145 - rnn_att - 36_r 20_r 13 - 11 36 41 6 8 20 21 31 

#     "../logs/2023-10-06/41/",  # 0.3144 - rnn_att - 36_r 13      - 11 36 41 6 20 21 31 31
#     "../logs/2023-10-06/27/",  # 0.3145 - rnn_att - 36_r 20_r 13 - 11 36 41 6 20 21 31
    
     # STRIDE 2

#     "../logs/2023-10-07/25/",  # 0.3139 - rnn_att - 36_r 13      - 11 36 41 6 20 21 31  stride 2
#     "../logs/2023-10-10/16/",  # 0.3140 - rnn_att - 36_r 13      - 11 36 41 6 20 21 31 31 refine

#     "../logs/2023-10-10/34/",  # 0.3125 - rnn_att - 36_r 13 25    - 11 36 41 6 20 21 31 31
#     "../logs/2023-10-10/35/",  # 0.3128 - rnn_att - 36_r 13 25    - 11 36 41 6 20 21 31 31  refine
    
#     "../logs/2023-10-10/37/",   # 0.3133 - rnn_att - 13 25    - 11 36 41 6 20 21 31 31  refine
#     "../logs/2023-10-10/38/",   # 0.3128 - rnn_att - 13 25    - 11 36 41 6 20 21 31 31 
#     
#     "../logs/2023-10-10/44/",   # 0.3118  - rnn_att - 36_r 13 25 27   - 11 36 41 6 20 21 31 31
#     "../logs/2023-10-10/47/",   # 0.3119  - rnn_att - 13 27   - 11 36 41 6 20 21 31 31   <--- 0.3096 with BEST
    
    # SUB 1 - 0.3096
    "../logs/2023-10-10/45/",   # 0.3111  - rnn_att - 36_r 25 27   - 11 36 41 6 20 21 31 31    <------ BEST LB 0.35
    "../logs/2023-10-10/42/",   # 0.3120  - rnn_att - 25 27   - 11 36 41 6 20 21 31 31
    "../logs/2023-10-10/46/",   # 0.3126  - rnn_att - 36_r 27   - 11 36 41 6 20 21 31 31
    
#     "../logs/2023-10-11/5/",  # 0.3130  - rnn_att - 36_r 25 27   - 11 6 21 31 31
#     "../logs/2023-10-12/22/",
#     "../logs/2023-10-12/24/",
    
    # SUB 2 - 0.3125
#     "../logs/2023-10-13/0/",
#     "../logs/2023-10-13/1/", 
]

In [ ]:
preds_oof = []
for exp_folder in EXP_FOLDERS:
    cfg = ConfigInf(json.load(open(exp_folder + "config.json", "r")))
    print(
        exp_folder, " --> ",
        "proba :",
        " - ".join(e[8:] for e, m in cfg.exp_folders if "proba" in m),
        "\t crop : ",
        " - ".join(e[8:] for e, m in cfg.exp_folders if "crop" in m),
    )

    df_oof, pred_oof = retrieve_preds(
        df_patient,
        df_img,
        cfg,
        exp_folder,
        custom_agg=False,
#         folds=[0]
    )
    preds_oof.append(pred_oof)
    
    losses, avg_loss = rsna_loss(pred_oof, df_oof)
    print(f'-> CV Score : {avg_loss :.4f}\n')

#     df_oof.to_csv(exp_folder + 'df_oof.csv', index=False)

pred_oof = np.mean(preds_oof, 0)

In [ ]:
losses, avg_loss = rsna_loss(pred_oof, df_oof)

for k, v in losses.items():
    print(f"- {k.split('_')[0][:8]} loss\t: {v:.3f}")

print(f'\n -> CV Score : {avg_loss :.4f}')

In [ ]:
# # Check kaggle score

# preds_oof = []
# for exp_folder in EXP_FOLDERS:
#     cfg = ConfigInf(json.load(open(exp_folder + "config.json", "r")))
#     print(
#         exp_folder, " --> ",
#         "proba :",
#         " - ".join(e[8:] for e, m in cfg.exp_folders if "proba" in m),
#         "\t crop : ",
#         " - ".join(e[8:] for e, m in cfg.exp_folders if "crop" in m),
#     )

#     df_oof, pred_oof = retrieve_preds(
#         df_patient,
#         df_img,
#         cfg,
#         exp_folder,
#         custom_agg=False,
#         folds=[0]
#     )
#     preds_oof.append(pred_oof)
    
# #     losses, avg_loss = rsna_loss(pred_oof, df_oof)
# #     print(f'-> CV Score : {avg_loss :.4f}')

# df_oof = df_oof.sort_values('patient_id').reset_index(drop=True).head(100)
# pred_oof = df_oof[df_oof.columns[-11:]].values

# losses, avg_loss = rsna_loss(pred_oof, df_oof)

# for k, v in losses.items():
#     print(f"- {k.split('_')[0][:8]} loss\t: {v:.3f}")

# print(f'\n -> CV Score : {avg_loss :.4f}')

# # y_pred = pd.read_csv('../output/sub4.csv').sort_values('patient_id').reset_index(drop=True)
# # y_pred.drop(["bowel_healthy", "extravasation_healthy"], axis=1, inplace=True)
# # df_oof[["patient_id"] + list(df_oof.columns[-11:])]

In [ ]:
# from inference.lvl2 import to_sub_format

# preds = pred_oof.copy()
# df = df_oof.copy()

# for i in range(preds.shape[1]):
#     df[f'pred_{i}'] = preds[:, i]

# # dfg = df.drop(['series', 'path', 'frame', 'patient_id'], axis=1).groupby('patient').mean().reset_index()
# sub = to_sub_format(df.rename(columns={"patient_id": "patient"}))
# sub.to_csv(f'../output/submission_{int(avg_loss * 1000)}.csv', index=False)

# print('-> Saved to', f'../output/submission_{int(avg_loss * 1000)}.csv')

# sub.head()

In [ ]:
# plt.figure(figsize=(22, 4))

# plt.subplot(1, 5, 1)
# plot_confusion_matrix(pred_oof[:, 0] > 0.5, df_oof[PATIENT_TARGETS[0]], display_labels=["ok", "injury"], normalize=None, show_label=True)
# plt.title(PATIENT_TARGETS[0])

# plt.subplot(1, 5, 2)
# plot_confusion_matrix(pred_oof[:, 1] > 0.5, df_oof[PATIENT_TARGETS[1]], display_labels=["ok", "injury"], normalize=None)
# plt.title(PATIENT_TARGETS[1])

# plt.subplot(1, 5, 3)
# plot_confusion_matrix(pred_oof[:, 2:5].argmax(-1), df_oof[PATIENT_TARGETS[2]], display_labels=["ok", "low", "high"], normalize=None)
# plt.title(PATIENT_TARGETS[2])

# plt.subplot(1, 5, 4)
# plot_confusion_matrix(pred_oof[:, 5:8].argmax(-1), df_oof[PATIENT_TARGETS[3]], display_labels=["ok", "low", "high"], normalize=None)
# plt.title(PATIENT_TARGETS[3])

# plt.subplot(1, 5, 5)
# plot_confusion_matrix(pred_oof[:, 9:].argmax(-1), df_oof[PATIENT_TARGETS[4]], display_labels=["ok", "low", "high"], normalize=None)
# plt.title(PATIENT_TARGETS[4])

# plt.show()

### Crop models

In [ ]:
# from data.preparation import get_df_series

# dfs = []
# for i in tqdm(range(4)):
#     preds = np.load(f'../logs/2023-09-30/20/pred_val_{i}.npy')
#     df_series = get_df_series(df_patient[df_patient['fold'] == i], df_img[df_img['fold'] == i])
    
#     cs = ['pred_healthy', 'pred_low', 'pred_high']
#     for i, c in enumerate(cs):
#         df_series[c] = preds[:, i]
    
#     df_series = df_series.groupby(["patient_id", "series"]).agg(list).reset_index()
    
#     i = 2
#     for idx, c in enumerate(["kidney", "liver", "spleen"]):
#         for p in ['pred_healthy', 'pred_low', 'pred_high']:
#             df_series[f"pred_{i}"] = np.array(df_series[p].values.tolist())[:, idx]
#             i += 1

#     df_series = df_series[["patient_id"] + [f"pred_{i}" for i in range(2, 11)]]
#     df = df_series.groupby("patient_id").mean().reset_index()
#     dfs.append(df)
    
# df_crop = pd.concat(dfs, ignore_index=True)

In [ ]:
# df_oof_c = df_oof.merge(df_crop, on="patient_id")

# pred_oof_1 = df_oof_c[["pred_0", "pred_1"] + [f"pred_{i}_x" for i in range(2, 11)]].values
# pred_oof_2 = df_oof_c[["pred_0", "pred_1"] + [f"pred_{i}_y" for i in range(2, 11)]].values

In [ ]:
# for i, c in enumerate(['bowel_injury', 'extravasation_injury', 'kidney_healthy', 'kidney_low', 'kidney_high', 'liver_healthy', 'liver_low', 'liver_high', 'spleen_healthy', 'spleen_low', 'spleen_high']):
#     if i > 1:
#         auc_1 = roc_auc_score(df_oof[c], pred_oof_1[:, i])
#         auc_2 = roc_auc_score(df_oof[c], pred_oof_2[:, i])
#         print(f"{c[:10]} AUC:\t {auc_1 :.3f} - {auc_2 :.3f}  ({'+' if auc_2 > auc_1 else ''}{auc_2 - auc_1 :.3f})")

### Rescaling

In [ ]:
rescaled_preds = []
df_oofs = []

print("Exp folders :\t", "\t".join(EXP_FOLDERS))

for fold in [0, 1, 2, 3, "fullfit"]:
    print(f'\n ->  Fold {fold}')
    train_folds = [f for f in range(4) if f != fold] 
    
    preds_oof = []
    for exp_folder in EXP_FOLDERS:
        df_oof, pred_oof = retrieve_preds(df_patient, df_img, Config, exp_folder, folds=train_folds)
        preds_oof.append(pred_oof)
    pred_oof = np.mean(preds_oof, 0)

    losses, avg_loss = rsna_loss(pred_oof, df_oof)
    print(f'\n- Train Score : {avg_loss :.3f}')
    
    print(f'\n- Optimize coeffs on folds {train_folds}:\n')
    pred_oof_ = pred_oof.copy()
    best_score = avg_loss
    factors = {i: [1, 0] for i in range(pred_oof.shape[1])}
    for _ in range(2):

        for i in range(pred_oof.shape[1]):
            scores = {}
            for factor in np.round(np.arange(0.5, 1.5, 0.1), 2):
                for shift in np.round(np.arange(-0.1, 0.11, 0.1), 2):
    #             for shift in [-0.1, 0, 0.1]:
                    pred_oof_r = pred_oof_.copy()
                    pred_oof_r[:, i] = pred_oof_r[:, i] * factor + shift
                    pred_oof_r[:, i] = np.clip(pred_oof_r[:, i], 0.00001, 0.99999)

                    losses, avg_loss = rsna_loss(pred_oof_r, df_oof)
                    scores[(factor, shift)] = avg_loss

        #     print(scores)
            best_coefs, best_loss = min(scores.items(), key=operator.itemgetter(1))
            delta = best_score - best_loss 
            if delta > 0.0001:
                pred_oof_[:, i] = np.clip(pred_oof_[:, i] * best_coefs[0] + best_coefs[1], 0.00001, 0.99999)
                best_score = best_loss
                print(f'{i} - {best_coefs}  -  {best_loss :.3f}  (-{delta:.4f})')
                factors[i][0] = np.round(best_coefs[0] * factors[i][0], 2)
                factors[i][1] += best_coefs[1]
                
    if fold != "fullfit":
        # Validate
        preds_oof = []
        for exp_folder in EXP_FOLDERS:
            df_oof, pred_oof = retrieve_preds(df_patient, df_img, Config, exp_folder, folds=[fold])
            preds_oof.append(pred_oof)

        pred_oof = np.mean(preds_oof, 0)

        losses, avg_loss = rsna_loss(pred_oof, df_oof)
        print(f'\n- Val Score : {avg_loss :.3f}')

        pred_oof_ = pred_oof.copy()
        for i in range(pred_oof.shape[1]):
            pred_oof_[:, i] *= factors[i][0]
            pred_oof_[:, i] += factors[i][1]
            pred_oof_[:, i] = np.clip(pred_oof_[:, i], 0.00001, 0.99999)

        losses, avg_loss = rsna_loss(pred_oof_, df_oof)
        print(f'- Rescaled val Score : {avg_loss :.3f}')

        rescaled_preds.append(pred_oof_)
        df_oofs.append(df_oof)

#     break

In [ ]:
# losses, avg_loss = rsna_loss(np.concatenate(rescaled_preds), pd.concat(df_oofs))

# for k, v in losses.items():
#     print(f"- {k.split('_')[0][:8]} loss\t: {v:.3f}")
    
# print(f'\n -> CV Score : {avg_loss :.3f}')

In [ ]:
# from inference.lvl2 import to_sub_format

# preds = np.concatenate(rescaled_preds).astype(np.float64)
# df = pd.concat(df_oofs)

# for i in range(preds.shape[1]):
#     df[f'pred_{i}'] = preds[:, i]

# # dfg = df.drop(['series', 'path', 'frame', 'patient_id'], axis=1).groupby('patient').mean().reset_index()
# sub = to_sub_format(df.rename(columns={"patient_id": "patient"}))
# sub.to_csv(f'../output/submission_{int(avg_loss * 1000)}.csv', index=False)

# print('-> Saved to', f'../output/submission_{int(avg_loss * 1000)}.csv')

# sub.head()

Done ! 